Loading here all our modules

In [ ]:
import pathlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

import gizmo_analysis as ga
import ananke as an

Configuring matplotlib

In [ ]:
plt.rcdefaults()
plt.rc('font', size=17)
plt.rc('axes', unicode_minus=False)
%matplotlib inline

Defining here some parameters

In [ ]:
# simulation specific
simulation = 'm11h_res7100'
redshift = 0.

# plotting specific
cmap = 'bone'
hex_cmap = 'cividis'
kpc_aperture = 30
kpc_pixel = 0.5
sky_pix_deg = 1

# ananke specific
fsample = 0.3

# systemm specific
latte_directory = pathlib.Path('/home/altair/Data/latte')

# don't change
sim_dir = latte_directory / simulation
cmap = plt.get_cmap(cmap)
hex_cmap = plt.get_cmap(hex_cmap)

Read in star particles from the selected `latte` snapshot

In [ ]:
part = ga.io.Read.read_snapshots(species='star',
                                 snapshot_value_kind='redshift',
                                 snapshot_values=redshift,
                                 simulation_directory=str(sim_dir.resolve()),
                                 elements='all',
                                 assign_hosts=True,
                                 assign_hosts_rotation=True,
                                 assign_orbits=True)

Let's have a look at the particle data in the host principal axes -> (x,y,z) = (major,intermediate,minor)

In [ ]:
# array of positions in principal axes (in kpc)
pos_pa = part['star'].prop('host.distance.principal')

# preparing plot
bins = np.linspace(-kpc_aperture, kpc_aperture, int(np.ceil(2*kpc_aperture/kpc_pixel)+1))
fig = plt.figure(figsize=(20,8))
fig.subplots_adjust(wspace=0., hspace=0.)
_temp = np.array([2*[0],2*[1]])
_temp = np.dstack([_temp,_temp.T]).reshape((4,2))[:-1]
_temp = np.vstack([_temp, _temp+[0,3]])
axs = [plt.subplot2grid(shape=(2, 5), loc=loc, rowspan=1, colspan=1) for loc in _temp]
for ax in axs[1:]: ax.sharex(axs[0]), ax.sharey(axs[0])
axs[0].xaxis.set_tick_params(labeltop=True, labelbottom=False)
axs[1].xaxis.set_tick_params(labeltop=True, labelbottom=True)
axs[1].yaxis.set_tick_params(labelleft=False, labelright=True)
axs[0].xaxis.set_label_position('top')
axs[1].yaxis.set_label_position('right')
axs[1].xaxis.set_label_position('top')
axs[3].xaxis.set_tick_params(labeltop=True, labelbottom=False)
axs[4].xaxis.set_tick_params(labeltop=True, labelbottom=True)
axs[4].yaxis.set_tick_params(labelleft=False, labelright=True)
axs[3].xaxis.set_label_position('top')
axs[4].yaxis.set_label_position('right')
axs[4].xaxis.set_label_position('top')

# hide some axes for now
for ax in axs[3:]: ax.set_visible(False)

# plotting 2d histograms along each 3 principal axes
_,_,_,h2d = axs[0].hist2d(pos_pa[:,0], pos_pa[:,2], bins=[bins,bins], norm=LogNorm(), cmap=cmap)
vmin,vmax = h2d.get_clim()
_,_,_,h2d = axs[1].hist2d(pos_pa[:,1], pos_pa[:,2], bins=[bins,bins], norm=LogNorm(vmin=vmin, vmax=vmax), cmap=cmap)
_,_,_,h2d = axs[2].hist2d(pos_pa[:,0], pos_pa[:,1], bins=[bins,bins], norm=LogNorm(vmin=vmin, vmax=vmax), cmap=cmap)

# setting up all the labels
axs[2].set_xlabel('X [kpc]')
axs[2].set_ylabel('Y [kpc]')
axs[1].set_xlabel('Y [kpc]')
axs[1].set_ylabel('Z [kpc]')
axs[0].set_ylabel('Z [kpc]')
axs[0].set_xlabel('X [kpc]')

# detailing final subplots
axs[2].set_aspect(1.0/axs[2].get_data_ratio())
axs[0].set_facecolor(cmap(0))
axs[1].set_facecolor(cmap(0))
axs[2].set_facecolor(cmap(0))

# setting up the colorbar
ax0color_axis = axs[2].inset_axes([1, 0., 0.03, 1.], transform=axs[2].transAxes)
ax0cbar = fig.colorbar(h2d, cax=ax0color_axis)
ax0cbar.set_label('Number of stars\nper pixel')


Defining some other star particle data

In [ ]:
# array of velocities in principal axes (in km/s)
vel_pa = part['star'].prop('host.velocity.principal')

# array of star particle mass in solar masses
mass = part['star']['mass']

# array of decimal log stellar ages (in Gyr)
log_age = np.log10(part['star'].prop('age') * 1e9)

# array of star particle metallicities
feh = part['star'].prop('metallicity.fe')

abundances_list = ['helium', 'carbon', 'nitrogen', 'oxygen', 'neon', 'magnesium', 'silicon', 'sulfur', 'calcium']
# dictionary of chemical abundance arrays (X/H)
abundances = {'sulphur' if el == 'sulfur' else el: part['star'].prop('metallicity.' + el) for el in abundances_list}

# alpha abundance (Mg/Fe)
alpha = abundances['magnesium'] - feh

Preparing the star particles data to be used by `ananke`, masking only a sphere within 30 kpc

In [ ]:
mask = np.linalg.norm(pos_pa, axis=1)<=30

p = {}
p['pos3'] = pos_pa[mask]    # position in kpc
p['vel3'] = vel_pa[mask]    # velocity in km/s
p['mass'] = mass[mask]      # mass in solar masses
p['age'] = log_age[mask]    # log age in Gyr
p['feh'] = feh[mask]        # [Fe/H]
for el, abun in abundances.items():  
    p[el] = abun[mask]      # other abundances as [X/H]

p['alpha'] = alpha[mask]    # alpha abundance [Mg/Fe]

p['parentid'] = np.where(mask)[0]           # indices of parent particles in snapshot
p['dform'] = 0*p['mass']  # dummy variable for now

Now we can prepare the `ananke` surveyor. Default surveyor is set to simulate a Roman + HST photometric system.

In [ ]:
surveyor = an.Ananke(p, name='anankethon', fsample=fsample)

Let's make the surveyor survey!

In [ ]:
survey = surveyor.run()

Due to the large memory footprint potential of the survey data, the output object interfaces a `vaex` memory-mapped dataframe

In [ ]:
survey

In [ ]:
type(survey)

Currently, accessing the underlying `vaex` dataframe can be done directly via the following property (planning to improve that in the future)

In [ ]:
survey._vaex

In [ ]:
type(survey._vaex)

That said, some operations are directly accessible from the output object

In [ ]:
survey[['px','py','pz']]

In [ ]:
survey.teff

To access the mock catalog bands for the chosen photometric system, they are selectable with the following string key syntax `f"{name_of_photometric_system}_{name_of_band}"` (that syntax might change in the future to `f"{name_of_photometric_system}/{name_of_band}"`, I'll raise a `DeprecationWarning` for a few version when that's effectively planned)

In [ ]:
survey['wfirst-hst_f814w']

In [ ]:
survey['wfirst-hst_h158']

In [ ]:
survey[['wfirst-hst_z087','wfirst-hst_y106','wfirst-hst_j129','wfirst-hst_w149','wfirst-hst_h158','wfirst-hst_f184']]

Let's have a look at the population of synthetic star we created

In [ ]:
# show axes that were hidden before 
for ax in axs[3:]: ax.set_visible(True)

# converting vaex sub-dataframe to numpy
mock_pos = survey[['px','py','pz']].to_pandas_df().to_numpy()

# plotting 2d histograms along each 3 principal axes
_,_,_,h2d = axs[3].hist2d(mock_pos[:,0], mock_pos[:,2], bins=[bins,bins], norm=LogNorm(), cmap=cmap)
vmin,vmax = h2d.get_clim()
_,_,_,h2d = axs[4].hist2d(mock_pos[:,1], mock_pos[:,2], bins=[bins,bins], norm=LogNorm(vmin=vmin, vmax=vmax), cmap=cmap)
_,_,_,h2d = axs[5].hist2d(mock_pos[:,0], mock_pos[:,1], bins=[bins,bins], norm=LogNorm(vmin=vmin, vmax=vmax), cmap=cmap)

# setting up all the labels
axs[5].set_xlabel('X [kpc]')
axs[5].set_ylabel('Y [kpc]')
axs[4].set_xlabel('Y [kpc]')
axs[4].set_ylabel('Z [kpc]')
axs[3].set_ylabel('Z [kpc]')
axs[3].set_xlabel('X [kpc]')

# detailing final subplots
axs[3].set_facecolor(cmap(0))
axs[4].set_facecolor(cmap(0))
axs[5].set_facecolor(cmap(0))

# setting up the colorbar
ax0color_axis = axs[5].inset_axes([1, 0., 0.03, 1.], transform=axs[5].transAxes)
ax0cbar = fig.colorbar(h2d, cax=ax0color_axis)
ax0cbar.set_label('Number of stars\nper pixel')

fig

While this is showing here the mock stars in their 3D coordinates, `ananke` also returns celestial coordinates

In [ ]:
ra_deg = survey.ra.to_numpy()
dec_deg = survey.dec.to_numpy()

fig2 = plt.figure(figsize=(20,8))
ax = fig2.add_subplot(111)

_,_,_,h2d = ax.hist2d(ra_deg, dec_deg, bins=[np.linspace(0,360,int(np.ceil(361/sky_pix_deg))),
                                             np.linspace(-90,90,int(np.ceil(181/sky_pix_deg)))],
                                       norm=LogNorm(), cmap=cmap)

# setting up all the labels
ax.set_xlabel('RA [deg]')
ax.set_ylabel('DEC [deg]')

# detailing final subplots
ax.set_facecolor(cmap(0))

# setting up the colorbar
ax0color_axis = ax.inset_axes([1, 0., 0.03, 1.], transform=ax.transAxes)
ax0cbar = fig.colorbar(h2d, cax=ax0color_axis)
ax0cbar.set_label('Number of stars\nper pixel')

In [ ]:
h2d.set_clim((100,h2d.get_clim()[1]))
fig2

Let's also have a look at some hertzsprung russell/color magnitude diagrams

In [ ]:
luminosity = survey.lum.to_numpy()
temperature = survey.teff.to_numpy()

fig3,axs=plt.subplots(nrows=1,ncols=3,figsize=(20,8), subplot_kw={'adjustable':'box'})
fig3.subplots_adjust(wspace=0.8)
for ax in axs[1:]: ax.set_visible(False)

hb = axs[0].hexbin(temperature, luminosity, gridsize=(241),
                   bins='log', cmap=hex_cmap)

axs[0].invert_xaxis()
axs[0].set_xlabel('log10(Temperature [Kelvin])')
axs[0].set_ylabel('log10(Luminosity [Lsun])')
axs[0].set_aspect(1.0/axs[0].get_data_ratio())

ax0color_axis = axs[0].inset_axes([1, 0., 0.03, 1.], transform=axs[0].transAxes)

ax0cbar = fig.colorbar(hb, cax=ax0color_axis, location='right')
ax0cbar.set_label('Number of stars\nper hexcell')


In [ ]:
for ax in axs[1:]: ax.set_visible(True)

mag_cmd1 = survey['wfirst-hst_f814w'].to_numpy()
color_cmd1 = survey['wfirst-hst_f555w'].to_numpy() - mag_cmd1

hb = axs[1].hexbin(color_cmd1, mag_cmd1, gridsize=(241),
                   bins='log', cmap=hex_cmap)

axs[1].invert_yaxis()
axs[1].set_xlabel('{HST}: F555W-F814W')
axs[1].set_ylabel('{HST}: F814W')
axs[1].set_aspect(1.0/axs[1].get_data_ratio())

ax1color_axis = axs[1].inset_axes([1, 0., 0.03, 1.], transform=axs[1].transAxes)

ax1cbar = fig.colorbar(hb, cax=ax1color_axis, location='right')
ax1cbar.set_label('Number of stars\nper hexcell')


mag_cmd2 = survey['wfirst-hst_f184'].to_numpy()
color_cmd2 = survey['wfirst-hst_z087'].to_numpy() - mag_cmd2

hb = axs[2].hexbin(color_cmd2, mag_cmd2, gridsize=(241),
                   bins='log', cmap=hex_cmap)

axs[2].invert_yaxis()
axs[2].set_xlabel('{Roman}: F087-F158')
axs[2].set_ylabel('{Roman}: F158')
axs[2].set_aspect(1.0/axs[2].get_data_ratio())

ax2color_axis = axs[2].inset_axes([1, 0., 0.03, 1.], transform=axs[2].transAxes)

ax2cbar = fig.colorbar(hb, cax=ax2color_axis, location='right')
ax2cbar.set_label('Number of stars\nper hexcell')

fig3


This photometric system isn't the only one natively built in `py-ananke`, there a couple of others although just a couple. We plan to add more in the future, this is work in progress for now. This command will return which can be used:

In [ ]:
# for now, it is necessary to import Galaxia's submodule
import Galaxia_ananke

Galaxia_ananke.photometry.available_photo_systems

That said, if you wish to add your own photometric system, this is currently possible but we hope to improve the API

In [ ]:
iso = Galaxia_ananke.photometry.available_photo_systems['padova/WFIRST-HST']

isochrone_data = {i_file.metallicity: i_file.data.to_pandas()[
    ['log(age/yr)', 'M_ini', 'M_act', 'logL/Lo', 'logTe', 'logG', 'Z087', 'Y106', 'J129', 'H158', 'F184', 'W149', 'F475W', 'F555W', 'F606W', 'F814W', 'F110W', 'F160W']
    ].rename(columns=dict(zip(['log(age/yr)', 'M_ini', 'M_act', 'logL/Lo', 'logTe', 'logG'],['Age','M_ini', 'M_act', 'Lum', 'T_eff', 'Grav'])))
    for i_file in iso.isochrone_files}


In [ ]:
np.array(list(isochrone_data.keys()))

In [ ]:
isochrone_data[0.008]

In [ ]:
# Galaxia_ananke.photometry.available_photo_systems.add_isochrone('Test', isochrone_data)

# new_iso = Galaxia_ananke.photometry.available_photo_systems['py_custom/Test']